                                                           (Pressione Ctrl + Click)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/169rmtyEwRg4YXrWxkpUfssPbWaAQpnxK)

No aprendizado de máquina e na estatística, a seleção de recursos, também conhecida como seleção de variáveis, seleção de atributos ou seleção de subconjuntos de variáveis, é o processo de seleção de um subconjunto de recursos relevantes (variáveis, preditores) para uso na construção do modelo.

 As técnicas de seleção de recursos são usadas por vários motivos:
     * simplificação de modelos para facilitar a interpretação por pesquisadores / usuários;
     * tempo de treinamento mais curto;
     * reduz o processo de overfitting

## O que seria selecionar parâmetros?

Parâmetros Selecionados com a utilização de filtros, podem ser usados como entrada de qualquer modelo de machine learning.

Os métodos se diferem aos tipos de dados:

Univariado   -> Análise de cada variável separadamente

Bivariado    -> Método de análise de duas variáveis (causa/efeito)

Multivariado -> Métodos de análise de relações de múltiplas variáveis dependentes ou independentes

## Qual o melhor Método?

Como tudo na vida, não existe o modelo perfeito mas o que se adequa melhor aos seus dados. Como cientista de dados, uma de nossas 
funções é testar o máximo de hipóteses para melhorar o desempenho geral do aprendizado, evitando o overfitting e estudando a relaçao
os dados sempre da melhor forma.

## Download dos Dados
* https://github.com/philipesantos136/Portfolio/tree/master/Feature-Selection

O Objetivo principal dessa base de dados é criar um modelo que prevê disponibilização de crédito para os clientesdo banco, os que tiveram como TARGET = 1, são os que preencheram o requisito de liberação de crédito.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold

In [ ]:
from sklearn import metrics
from imblearn.under_sampling import RandomUnderSampler
!pip install -q scikit-plot
import scikitplot as skplt

In [ ]:
data = pd.read_csv('santander.csv', nrows = 20000)
data.head()

In [ ]:
# checando a distribuição da variável alvo
print(data.TARGET.value_counts())
print("\nConseção de Crédito representam {:.2f}% Do Dataset.\n".format((data[data.TARGET == 1].shape[0] / data.shape[0]) * 100))

# plotando gráfico de barras
fig, ax = plt.subplots()
sns.countplot('TARGET', data=data, ax=ax)
ax.set_title('Distribuição Das Classes')
plt.plot()
plt.savefig('original_dist.png')

In [ ]:
X = data.drop('TARGET', axis = 1)
y = data['TARGET']

X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0, stratify = y)

## Balanceando os Dados

In [ ]:
# aplicando o UnderSampler
rus = RandomUnderSampler()
X_rus_train, y_rus_train = rus.fit_sample(X_train, y_train)

# checando o balanceamento das classes
print(pd.Series(y_rus_train).value_counts())

# plotando a nova distribuição das classes
sns_plot = sns.countplot(y_rus_train);
plt.savefig("balanced.png")

### Remoção de parâmetros constantes

In [ ]:
constant_filter = VarianceThreshold(threshold=0)
constant_filter.fit(X_rus_train)

In [ ]:
#Parâmetros constantes
constant_filter.get_support().sum()

In [ ]:
#Removendo os parâmetros constantes
X_train_filter = constant_filter.transform(X_rus_train)
X_test_filter = constant_filter.transform(X_test)

X_train_filter.shape, X_test_filter.shape, X_train.shape

## Remoção de Parâmetros Semi-Constantes

In [ ]:
quasi_constant_filter = VarianceThreshold(threshold=0.01)

In [ ]:
quasi_constant_filter.fit(X_train_filter)

In [ ]:
#Parâmetros Semi-constantes
quasi_constant_filter.get_support().sum()

In [ ]:
#Removendo os parâmetros Semi-constantes
X_train_quasi_filter = quasi_constant_filter.transform(X_train_filter)
X_test_quasi_filter = quasi_constant_filter.transform(X_test_filter)

X_train_quasi_filter.shape, X_test_quasi_filter.shape

## Parâmetros Duplicados

In [ ]:
X_train_T = X_train_quasi_filter.T
X_test_T = X_test_quasi_filter.T

In [ ]:
X_train_T = pd.DataFrame(X_train_T)
X_test_T = pd.DataFrame(X_test_T)

In [ ]:
X_train_T.shape, X_test_T.shape

In [ ]:
#Duplicados
X_train_T.duplicated().sum()

In [ ]:
duplicated_features = X_train_T.duplicated()

In [ ]:
#Selecionando os Parâmetros que devem ficar ou não
features_to_keep = [not index for index in duplicated_features]

In [ ]:
X_train_unique = X_train_T[features_to_keep].T
X_test_unique = X_test_T[features_to_keep].T

In [ ]:
X_train_unique.shape, X_train.shape

## Criando modelo e comparando as performances

In [ ]:
def run_randomForest(X_train, X_test, y_train, y_test):
    rf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    print("Classification Report")
    print(metrics.classification_report(y_test, y_pred, labels = [1, 0],digits=4))
    print()

In [ ]:
%%time
#Modelo com os dados balanceados
run_randomForest(X_train_unique, X_test_unique, y_rus_train, y_test)

In [ ]:
%%time
run_randomForest(X_train, X_test, y_train, y_test)

Fica fácil ver que os dados balanceados e com a seleção de parâmetros mais simples tiveram bons resultados, analisemos:
* A Precisão do modelo diminuiu para casos onde há concessão de crédito, tornando o modelo mais conservador.
* A Sensibilidade (Recall) ficou mais equilibrada e melhorou significativamente para os casos onde há conseção de crédito.

## O que isso quer dizer na prática?

    Com uma precisão menor para os casos positivos de concessão de crédito e um recall balanceado, o modelo será preciso tendo uma menor chance de conceder crédito a quem não deveria, mesmo que o cliente esteja enquadrado para receber o benefício. Isso é um bom resultado. Conceder crédito quando na verdade não deveria, pode ser uma grande perda de dinheiro.

# Conclusões

Ciência de Dados é um campo em expansão, boa parte do tempo de trabalho desse profissional é preparando os dados de maneira correta. Neste projeto, o intuito foi apresentar uma espécie de seleção de parâmetros para o estudo de concessão de crédito.

Como projeto inicial, apenas o método mais simples foi implementado, existem muitos outros, que, na maioria das vezes, se complementam.

Em próximos projetos, podemos implementar os outros modelos (Correlação de Person, X2, Informação Mútua, etc) para estudarmos o que acontece com os dados a cada etapa.

    Obrigado!

## Referências
* Gareth James; Daniela Witten; Trevor Hastie; Robert Tibshirani (2013). An Introduction to Statistical Learning. Springer. p. 204.
* (Burns, 2000; M. Hill & A. Hill, 2000; Thomas & Nelson, 1996). Sin.: Estatística Univariável (Dorsch et al., 2001); Estatística a uma dimensão (Dagnelie, n.d.).
* Estatística Bivariável (Dorsch et al., 2001); Estatística a duas dimensões (Dagnelie, n.d.)
* Estatística Multivariável (Dorsch et al., 2001); Estatística a três ou mais dimensões (Dagnelie, n.d.).

## Sites
* https://en.wikipedia.org/wiki/Feature_selection
* https://alexandreramos.blogs.sapo.pt/7901.html
* https://sigmoidal.ai/como-lidar-com-dados-desbalanceados/